# Features Extraction

Ja hem creat la base de dades amb la funció build_database() que podem trobar en el nostre repositori, aquesta funció ens ha seperat les imatges en imatges d'entrenament i imatges de validació i ha creat un arxiu anomenat ID.txt tan a la carpeta de train com val on es troben guardats l'ID de totes les imatges que partanyen a la carpeta en questió. A continuació definirem les funcions necessàries per extreure les caractarístiques de cada imatge:

## Get_params():

Creem aquesta funció per a definir alguns dels peràmetres que necessitarem per invocar la resta de funcions:

In [8]:
# -*- coding: utf-8 -*-

import os
import pandas as pd
import numpy as np

def get_params():

    '''
    Define dictionary with parameters
    '''
    params = {}

    # Source data
    params['root'] = r'/home/marta/Desktop/GDSA'
    params['database'] = 'TerrassaBuildings900'

    # To generate
    params['root_save'] = 'save'
    params['image_lists'] = 'image_lists'
    params['feats_dir'] = 'features'
    params['rankings_dir'] = 'rankings'
    params['classification_dir'] = 'classification'

    # Parameters
    params['split'] = 'val'
    params['descriptor_size'] = 1000
    params['descriptor_type'] = 'ORB'
    params['max_size'] = 800
    params['svm_tune'] =[{'C':[0.1, 1, 10, 100, 1000, 10000],'kernel':('linear', 'rbf'), 'gamma':[1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]}]
    # We read the training annotations to know the set of possible labels
    data = pd.read_csv(os.path.join(params['root'],params['database'],'train','annotation.txt'), sep='\t', header = 0)

    # Store them in the parameters dictionary for later use
    params['possible_labels'] = np.unique(data['ClassID'])

    create_dirs(params)

    return params


def make_dir(dir):
    '''
    Creates a directory if it does not exist
    dir: absolute path to directory to create
    '''
    if not os.path.isdir(dir):
        os.makedirs(dir)

def create_dirs(params):

    '''
    Create directories specified in params
    '''
    save_dir = os.path.join(params['root'], params['root_save'])

    make_dir(save_dir)
    make_dir(os.path.join(save_dir,params['image_lists']))
    make_dir(os.path.join(save_dir,params['feats_dir']))
    make_dir(os.path.join(save_dir,params['rankings_dir']))
    make_dir(os.path.join(save_dir,params['classification_dir']))

    make_dir(os.path.join(save_dir,params['rankings_dir'],params['descriptor_type']))
    make_dir(os.path.join(save_dir,params['rankings_dir'],params['descriptor_type'],params['split']))
    make_dir(os.path.join(save_dir,params['classification_dir'],params['descriptor_type']))

## Local_feature_extraction():

Amb aquesta funció extreiem els descriptors d'una imatge. Per a fer-ho ens hem decantat per utilitzar el métode ORB ja que és més eficient que SIFT i SURF. Per a no saturar el programa al compilar-ho reduim el tamany de les imatges que són massa grans i també hem assignat el numero de kyepoints a calcular a 2000, aquest numero pot ser modificat per obtenir resultats més acurats. Finalment la funció retornarà els descriptors detectats i calculats:

In [3]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import cv2
import math
from matplotlib import pyplot as plt


def local_feature_extraction(params,image):
    #llegim la imatge:
    img = cv2.imread(image)
    #Cambiem la mida de la imatge:
    if not img is None:
        img1 = resize_image(params,img)
        #linea que soluciona un bug de opencv
        cv2.ocl.setUseOpenCL(False)

        # Creem l'objecte ORB que tindrà 200k keypoints. (Perametre que podem modificar per no saturar el programa)
        orb = cv2.ORB_create(2000)

        # Detectem els keypoints:
        kp = orb.detect(img1,None)

        # Calculem els descriptors amb els keypoints trobats.
        kp, des = orb.compute(img1, kp)

        # la sortida de la funció serà els descriptors
        return des



#Definim la funció resize_image per cambiar la mida de les imatges massa grans mantenint
#les proporcions (aspect ratio) height/width.
def resize_image(params,img):
    #Obtenim les dimensions de l'imatge amb la funció de numpy, shape.
    height, width = img.shape[:2]

    # En cas que la mida de la imatge sigui més petita que el parametre max_size
    #(mida maxima que especifiquem al invocar la funció), manté la mida width original.
    #Si la mida width de la imatge és més gran, passa a ser de la mida max_size.
    resize_dim = min(params['max_size'],width)
    #Per no perdre la relació d'aspecte normalitzem ara l'altura height segons la
    #relació entre l'amplada obtinguda i l'amplada original, per evitar numeros decimals arrodonim amb ceil
    dim = (resize_dim, math.ceil(height * resize_dim/width))

    img2=cv2.resize(img,dim)
    # La funció retorna una imatge nova amb les noves dimensions.
    return img2
    

## Train_codebook():

Amb aquesta funció calculem on es trobaran els centroids donat el conjunt de descriptors obtinguts i el número total de clusters que volem, el resultat d'aquest calcul és el kMeans que retorna la funció:

In [5]:
# -*- coding: utf-8 -*-
#funció que codifica els descriptors obtinguts en la local_feature_extraction i els assigna el centroid més pròxim
#que determinarà el cluster al que es troba el descriptor en la funció get_assignments.
from sklearn.cluster import MiniBatchKMeans

def train_codebook(descriptors,clusters):
    kMeans = MiniBatchKMeans(clusters)

    #funció que calcula els centroids agrupant-los en mini lots. Defineix els centroids amb un entrenament segons els 
    #descriptors i el numero de clusters (paraules) que volem.
    kMeans.fit(descriptors)
    return kMeans

## Get_Assignments():

Un cop calculats els centroids ara toca assignar per a cada descriptor a quin cluster pertany, recordem que el descriptor pertany al cluster el centroid del qual es troba més pròxim a aquest. Utilitzarem doncs el resultat de train_codebook() i els descriptors per a fer la predicció. Aquesta predicció és la que retorna la funció:

In [6]:
# -*- coding: utf-8 -*-
# Amb aquesta funció cada descriptor local (obtinguts anteriorment) s'assigna al
# cluster que li correspon (per a fer-ho es mira quin és el centroid més pròxim).
# Cal recordar que descriptors és una matriu amb forma=(n_samples,n_features)
def get_assignments (KMeans, descriptors): #kmeans és la sortida del codebook
    #Amb la funció predict podem calcular l'index del cluster per a cada mostra.
    assignments=KMeans.predict(descriptors)
    #Retornem el vector amb els index assignats per cada descriptor
    return assignments

## Build_Bow():

Finalment crearem el Bag of Words, que no és més que un histograma "vector" que suma +1 a 'x' cada cop que un descriptor te associat l'índex 'x' de cluster, dit d'altre manera, l'eix de cordenades serà per els índex de cluster (1, fins a n_clusters) i l'eix d'ordenades per a la quantitat de descriptors i així construirem l'histograma. La funció retornarà doncs una vector on cada posició correspondrà a un index de cluster i el numerò associat a la posició serà la quantitat normalitzada de descriptors associats a aquest:

In [9]:
# -*- coding: utf-8 -*-
import numpy as np
from sklearn import preprocessing

#assignments és la sortida de la funció get_assignments que assigna un index de cluster a cada descriptor.
def build_bow(assignments,kMeans): #kmeans és la sortida de la funció train_codebook (assigna el centroid a un descriptor)
    # Definim el vector "l'histograma" bag of words per a que tingui la mateixa mida que el numero de clusters que tinguem.
    bow = np.zeros(np.shape(kMeans.cluster_centers_)[0]) #cluster_centers_ és una matriu [n_clusters, n_features]
    # Assigments retorna un vector amb l'idex de cluster assignat a cada descriptor.
    # Construim un histograma sumant +1 per a cada descriptor assignat a un index de cluster.
    for a in assignments:
        bow[a] += 1
    #linea per arreglar un warning
    bow = np.float64(np.reshape(bow, (1,-1)))
    # És important normalitzar amb L2
    bow = preprocessing.normalize(bow)
    return bow

## Get_features():

Aquesta funció engloba tota la resta, ara es tracta de generar un arxiu Features.p que contingui per a cada imatge el seu BoW corresponent, recorrerem per als dos sets d'imatges que tenim (train i val) totes les funcions vistes anteriorment en ordre per a aconseguir finalment generar aquest fitxer:

In [14]:
# -*- coding: cp1252 -*-
import os, sys
import numpy as np
import pickle as pk
from local_feature_extraction import local_feature_extraction
from train_codebook import train_codebook
from get_assignments import get_assignments
from build_bow import build_bow
from get_params import get_params



def get_features(params):
    #-------- Imatges d'entrenament --------#

    #Obrim el fitxer que conte les ID de les imatges d'entrenament
    ID=open(os.path.join(params['root'],params['database'],'train','ID.txt'), 'r')
    #Extraccio de les caracteristiques de la imatge de la primera linia del ID.txt, la funcio readline() llegeix
    #una sola linea d'un fitxer, aquesta lectura llegeix un caràcter "\n" al final de la linea, menys quan arriva al
    #final del fitxer (l'absencia d'aquest indica el final) però com que no ens interessa tenir el caràcter al final
    #del nom, emperem la subfunció .replace() que substituirà aquest caràcter per un espai en blanc ' '.
    nom=str(ID.readline()).replace('\n','')
    #invoquem la funciṕ local_feature_extraction per la primera imatge que hem llegit guardada a "nom".
    #Amb os.path.join() obtenim la imatge del directori.
    if os.path.exists(os.path.join(params['root'],params['database'],'train','images', nom + '.jpg')):
        image=os.path.join(params['root'],params['database'],'train','images', nom + '.jpg')
    if os.path.exists(os.path.join(params['root'],params['database'],'train','images', nom + '.JPG')):
        image=os.path.join(params['root'],params['database'],'train','images', nom + '.JPG')

    des_train=local_feature_extraction(params,image)
    #Creem un diccionary amb la funció dic() on i guardarem els descriptors per a cada imatge.
    # A diferència de les seqüències que estan indexades per un rang de numeros,els diccionaris són com matrius que estàn
    #indexats per keys les quals poden ser de qualsevol tipus immutable: strings i numeros poden ser sempre keys.
    dic_train=dict()
    #Guardem per a la primera imatge amb ID "nom", els seus descriptors associats en el diccionari per a imatges d'entrenament:
    dic_train[nom]=des_train
    #Generem un bucle per fer el mateix amb la resta d'imatges d'entrenament.
    for line in ID:
        nom=str(line).replace('\n','')

        if os.path.exists(os.path.join(params['root'],params['database'],'train','images', nom + '.jpg')):
            image=os.path.join(params['root'],params['database'],'train','images', nom + '.jpg')
        if os.path.exists(os.path.join(params['root'],params['database'],'train','images', nom + '.JPG')):
            image=os.path.join(params['root'],params['database'],'train','images', nom + '.JPG')

        x=local_feature_extraction(params,image)
        #Creem un vector que contindrà tots els descriptors
        des_train=np.concatenate((des_train,x))
        #Per a cada imatge tindrem a la matriu els seus descriptors.
        dic_train[nom]=x
    #Tanquem el fitxer
    ID.close()

    #Calculem els centroids "entrenant" la funció KMeans amb el numero de clusters(paraules) que volem i amb els descriptors
    #de les imatges d'entrenament.
    clusters=100
    codebook=train_codebook(params,des_train,clusters) #des_train conté tots els descriptors de cada imatge concatenats
    #Obrim el fitxer que conte les ID de les imatges d'entrenament per poder llegirlo altre cop des de l'inici:
    ID=open(os.path.join(params['root'],params['database'],'train','ID.txt'), 'r')

    for line in ID:
        nom=str(line).replace('\n','')
        #Calculem els index de cluster per cada mostra. Utilitzem el diccionari creat anteriorment per obtenir els descriptors
        #per a cada imatge i els centroids definits amb la funció train_codebook:
        assignments=get_assignments(codebook,dic_train[nom])
        #Substituim per a cada imatge l'assignació que tenia en el diccionari per un histograma(Bow) que indicarà quants descriptors
        #hi ha per cada paraula(cluster):
        dic_train[nom]=build_bow(assignments,codebook)
    #Tanquem el fitxer
    ID.close()

    #Guardem el diccionari amb el BoW per cada imatge d'entrenament en l'arxiu "Features.txt".
    bow_train = open(os.path.join(params['root'],params['database'],'train','Features.p'), 'wb')
    #la funció dump() permet guardar la variable dic_train en l'arxiu features.txt indicat per la variable bow_train.
    pk.dump(dic_train,bow_train)
    bow_train.close() #Tanquem el fitxer features.txt
    feat=open(os.path.join(params['root'],params['database'],'train','Features.p'), 'rb')
    p = pk.load(feat)
    feat.close()

    #---------Imatges de validació ---------#

    #Fem els mateixos passos anteriors però per les imatges de validació:
    #Obrim el fitxer que conté les ID de les imatges de validacio
    ID = open(os.path.join(params['root'],params['database'],'val','ID.txt'), 'r')
    nom=str(ID.readline()).replace('\n','')

    if os.path.exists(os.path.join(params['root'],params['database'],'val','images', nom + '.jpg')):
        image=os.path.join(params['root'],params['database'],'val','images', nom + '.jpg')
    if os.path.exists(os.path.join(params['root'],params['database'],'val','images', nom + '.JPG')):
        image=os.path.join(params['root'],params['database'],'val','images', nom + '.JPG')

    des_val=local_feature_extraction(params,image)
    #Creacio del diccionari de les imatges de validacio
    dic_val=dict()
    dic_val[nom]=des_val
    for line in ID:
        nom=str(line).replace('\n','')
        if os.path.exists(os.path.join(params['root'],params['database'],'val','images', nom + '.jpg')):
            image=os.path.join(params['root'],params['database'],'val','images', nom + '.jpg')
        if os.path.exists(os.path.join(params['root'],params['database'],'val','images', nom + '.JPG')):
            image=os.path.join(params['root'],params['database'],'val','images', nom + '.JPG')
        #Extraccio de les caracteristiques (keypoints) per a les imatges de validacio
        x=local_feature_extraction(params,image)
        des_val=np.concatenate((des_val,x)) #Creem un vector que contè tots els descriptors
        #Omplim el diccionari amb els descriptors per cada imatge.
        dic_val[nom]=x
    #Tanquem el fitxer
    ID.close()

    #Amb el mateix numero de clusters definits anteriorment i amb el nou vector de descriptors per les imatges de validació entrenem
    #la funció KMeans per a que creï els centroids.
    codebook=train_codebook(params,des_val,clusters)
    #Obrim el fitxer que conte les ID de les imatges de validació per poder llegirlo altre cop des de l'inici:
    ID=open(os.path.join(params['root'],params['database'],'val','ID.txt'), 'r')
    for line in ID:
        nom=str(line).replace('\n','')
        #Calculem els index de cluster per cada mostra. Utilitzem el diccionari creat anteriorment per obtenir els descriptors
        #per a cada imatge i els centroids definits amb la funció train_codebook:
        assignments=get_assignments(codebook,dic_val[nom])
        #Substituim per a cada imatge l'assignació que tenia en el diccionari per un histograma(Bow) que indicarà quants descriptors
        #hi ha per cada paraula(cluster):
        dic_val[nom]=build_bow(assignments,codebook)
    #Tanquem el fitxer
    ID.close()

    #Guardem el diccionari amb el BoW de les imatges de validació en l'arxiu "Features.txt"
    bow_val = open(os.path.join(params['root'],params['database'],'val','Features.p'), 'wb')
    pk.dump(dic_val,bow_val)
    bow_val.close()

    feat=open(os.path.join(params['root'],params['database'],'val','Features.p'), 'rb')
    p = pk.load(feat)
    feat.close()
    print (p) # per observar el resultat
    
#Per a comprobar el funcionament de la funció:
params=get_params()
get_features(params)

{'17156-3055-5516': array([[ 0.1031691 ,  0.09419787,  0.13008278,  0.01794245,  0.09868349,
         0.1031691 ,  0.15251084,  0.09419787,  0.07625542,  0.10765471,
         0.09868349,  0.02691368,  0.08522665,  0.13456839,  0.11662594,
         0.08074103,  0.04037052,  0.19288136,  0.07625542,  0.08971226,
         0.13008278,  0.12111155,  0.19736697,  0.15251084,  0.09868349,
         0.15251084,  0.04485613,  0.07625542,  0.05831297,  0.05831297,
         0.14353962,  0.08971226,  0.16148207,  0.08074103,  0.06279858,
         0.15699646,  0.14353962,  0.01345684,  0.0358849 ,  0.05831297,
         0.04037052,  0.05382736,  0.08971226,  0.04934174,  0.12111155,
         0.11214033,  0.08971226,  0.08074103,  0.1704533 ,  0.08971226,
         0.17493891,  0.08074103,  0.06279858,  0.08971226,  0.16148207,
         0.08522665,  0.07176981,  0.0358849 ,  0.11662594,  0.12559717,
         0.1031691 ,  0.18839575,  0.0672842 ,  0.12559717,  0.0358849 ,
         0.21530943,  0.0986834

L'arxiu Features.p resultant contè la sortida anterior (pel cas de les imatges de validació que són les que hem mostrat), per entendrel millor observem el següent:

#### {'19079-24541-20311':
    array([[ 0.06526682,  0.05594299,  0.0885764 ,  0.12587173,  0.05594299,
         0.07459066,  0.0885764 ,  0.09323832,  0.10722407,  0.03729533,
         0.05594299,  0.1165479 ,  0.1165479 ,  0.12587173,  0.10722407,
         0.13985748,  0.05128108,  0.0885764 ,  0.1165479 ,  0.10256215,
         0.1165479 ,  0.21444814,  0.09323832,  0.13519556,  0.08391449,
         0.11188598,  0.15384323,  0.07459066,  0.11188598,  0.0279715 ,
         0.04661916,  0.06060491,  0.09790024,  0.13985748,  0.13053365,
         0.07925257,  0.07925257,  0.09790024,  0.10722407,  0.0885764 ,
         0.05128108,  0.13519556,  0.0885764 ,  0.07459066,  0.06060491,
         0.06526682,  0.06992874,  0.0885764 ,  0.09323832,  0.04661916,
         0.03729533,  0.11188598,  0.03729533,  0.15850514,  0.12587173,
         0.04195724,  0.13519556,  0.16316706,  0.10256215,  0.05128108,
         0.12587173,  0.06060491,  0.10256215,  0.02330958,  0.06060491,
         0.06992874,  0.0885764 ,  0.10722407,  0.12587173,  0.09790024,
         0.13519556,  0.15384323,  0.1445194 ,  0.06992874,  0.12587173,
         0.1165479 ,  0.13519556,  0.13985748,  0.0279715 ,  0.06992874,
         0.16316706,  0.06526682,  0.10722407,  0.06060491,  0.05128108,
         0.05594299,  0.08391449,  0.15850514,  0.06526682,  0.08391449,
         0.1165479 ,  0.07459066,  0.0885764 ,  0.09790024,  0.09323832,
         0.09790024,  0.06992874,  0.06992874,  0.09323832,  0.04195724]]), 
#### '16714-19434-30572': 
    array([[ 0.0724286 ,  0.0724286 ,  0.15391077,  0.07695538,  0.08600896,
         0.12675004,  0.13127683,  0.0724286 ,  0.0724286 ,  0.06790181, .... etc.

El número en negreta correspon a l'ID de cada imatge i el vector que segueix és l'histograma Bow d'aquesta imatge, cada posició correspon a un índex de cluster i el valor que té és el numerò de descriptors normalitzat associats a l'índex en questió.